In [1]:
from pycocotools.coco import COCO
import os
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from sentence_transformers import SentenceTransformer
from PIL import Image
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline, set_seed


# drive.mount('/content/drive')


/Users/andreu/Documents/Bachelors Degree In Computer Engineering/ThirdYear/ThirdTrimester/Deep Learning/Practices/Image2Text-UPF/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "../data/"
# data_dir = "/content/drive/MyDrive/Colab Notebooks/test/"
data_type = "val2014"
instances_ann_file = os.path.join(data_dir, f"instances_{data_type}.json")


coco = COCO(instances_ann_file)
# initialize COCO API for caption annotations
captions_ann_file = os.path.join(data_dir,  f"captions_{data_type}.json")
coco_caps = COCO(captions_ann_file)


loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../data/instances_val2014.json'

In [ ]:

# get image ids
ids = list(coco.anns.keys())

# pick a random image and obtain the corresponding URL
ann_id = np.random.choice(ids)
img_id = coco.anns[ann_id]["image_id"]
img = coco.loadImgs(img_id)[0]
url = img["coco_url"]

print(img)
# print URL and visualize corresponding image
print(url)
I = io.imread(url)
plt.axis("off")
plt.imshow(I)

# load and display captions
ann_ids = coco_caps.getAnnIds(imgIds=img["id"])
anns = coco_caps.loadAnns(ann_ids)
coco_caps.showAnns(anns)
plt.show()

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(Image.fromarray(I))
input_batch = input_tensor.unsqueeze(0)
model.classifier[6] = nn.Linear(in_features=4096, out_features=384, bias=True)
print(model)

with torch.no_grad():
    output = model(input_batch)

In [ ]:
print(output)

In [ ]:
model_name = 'gpt2'  # or 'gpt2-medium', 'gpt2-large', etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
input_text = "Hello my name is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
print(input_ids)
output = model.generate(input_ids)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(generated_text)
